# Exercício 1 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

Este exercício é dirigido às provas de segurança do algoritmo acima.

1. Construa um FOTS $\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$ usando este modelo nos inteiros. 
2. Considere como propriedade de segurança
                        `safety = (r > 0) and (r < N) and (r = a*s + b*t)`
    Prove usando $k$-indução que esta propriedade se verifica em qualquer traço do FOTS
3. Prove usando “Model-Checking” com interpelantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.


# Exercício 1 - Solução

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

In [2]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

In [3]:
def init(state, a, b, N):

    # Pre:
    A = Equals(state['pc'], Int(0))
    B = GT(Int(a), Int(0))
    C = GT(Int(b), Int(0))
    D = GT(Int(N), Int(0))
    E = GT(Int(N), Int(a))
    F = GT(Int(N), Int(b))
    G = And(Equals(state['r'], Int(a)),
        Equals(state['r_prime'], Int(b)),
        Equals(state['s'], Int(1)),
        Equals(state['s_prime'], Int(0)),
        Equals(state['t'], Int(0)),
        Equals(state['t_prime'], Int(1)),
           Equals(state['q'], Int(0)))
    return And(A,B,C,D,E,F, G) 
    
def error(state):
    return LT(state['r'], Int(0))

def trans(curr, prox, a, b):
           
    t01 = And(
        Equals(curr['pc'], Int(0)),
        Equals(prox['pc'], Int(1)),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prime'], prox['r_prime']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prime'], prox['s_prime']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prime'], prox['t_prime']),
        Equals(curr['q'], prox['q'])
    )
    
    t14 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r_prime'], Int(0)),
        Equals(prox['r_prime'], curr['r_prime']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prime'], prox['s_prime']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prime'], prox['t_prime'])
    )

    t12 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(2)),
        Not(Equals(curr['r_prime'], Int(0))),
        Equals(curr['r_prime'], prox['r_prime']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prime'], prox['s_prime']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prime'], prox['t_prime']),
        Equals(curr['q'], prox['q'])
    )
        
    quotient = 0
    r = curr['r']
    r_prime = curr['r_prime']
    
    while r >= r_prime:
        r -= r_prime
        quotient += 1
    division = quotient
    
    t23 = And(
        Equals(curr['pc'], Int(2)),
        Equals(prox['pc'], Int(3)),
        Equals(prox['q'], division),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prime'], prox['r_prime']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prime'], prox['s_prime']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prime'], prox['t_prime'])
    )
    
    times_q_r_prox = Times(curr['q'], curr['r_prime'])
    times_q_s_prox = Times(curr['q'], curr['s_prime'])
    times_q_t_prox = Times(curr['q'], curr['t_prime'])
    
    minus_r_times = Minus(curr['r'], times_q_r_prox)
    minus_s_times = Minus(curr['s'], times_q_s_prox)
    minus_t_times = Minus(curr['t'], times_q_t_prox)

    t31 = And(
        Equals(curr['pc'], Int(3)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['r'], curr['r_prime']),
        Equals(prox['r_prime'], minus_r_times),
        Equals(prox['s'], curr['s_prime']),
        Equals(prox['s_prime'], minus_s_times),
        Equals(prox['t'], curr['t_prime']),
        Equals(prox['t_prime'], minus_t_times),
        Equals(prox['q'], curr['q'])
    )
    
    t44 = And(
        Equals(curr['pc'], Int(4)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prime'], prox['r_prime']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prime'], prox['s_prime']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prime'], prox['t_prime']),
        Equals(curr['q'], prox['q'])
    )
    
    return Or(t01, t14, t12, t23, t31, t44)

In [4]:
def genTrace(vars,init,trans,error,n, a, b, N):
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0], a, b , N)
        Tks = [trans(X[i],X[i+1], a, b) for i in range(n)]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n+1):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
            # OUTPUT
            if (s.get_value(X[-1]['r_prime']).constant_value() == 0 and s.get_value(X[-1]['pc']).constant_value() == 4):
                print(f"r = {s.get_value(X[-1]['r'])}, s = {s.get_value(X[-1]['s'])}, t = {s.get_value(X[-1]['t'])}")
        else:
            print("not sat")

In [5]:
def safety(state, a, b, N):
    A = GT(state['r'], Int(0))
    B = GT(Int(N), state['r'])
    C = Equals(state['r'], Plus(Times(Int(a), state['s']), Times(Int(b), state['t'])))
    return And(A, B, C)

def stronger(state, a,b, N):
    return And(safety(state,a,b,N), init(state,a,b,N));

In [6]:
def kinduction_always(vars, init, trans, error, n, a, b, N, inv):
    with Solver(name="z3") as solver:
        X = [genState(vars, 'X', i) for i in range(n+1)]
        I = init(X[0], a, b, N)
        solver.add_assertion(I)
        
        for i in range(n-1):
            solver.add_assertion(trans(X[i], X[i+1], a, b))
        
        for i in range(n):
            solver.push()
            solver.add_assertion(Not(inv(X[i], a, b, N)))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                for i, state in enumerate(X):
                    print(f"> State {i}: pc = {solver.get_value(state['pc'])}\nq = {solver.get_value(state['q'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prime'])}\nt' = {solver.get_value(state['t_prime'])}\nr' = {solver.get_value(state['r_prime'])}")
                return
            solver.pop()
        
        X2 = [genState(vars, 'X', i+n) for i in range(n+1)]
        
        for i in range(n):
            solver.add_assertion(inv(X2[i], a, b, N))
            solver.add_assertion(trans(X2[i],X2[i+1], a, b))
        
        solver.add_assertion(Not(inv(X2[-1], a, b, N)))
        
        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            for i, state in enumerate(X):
                print(f"> State {i}: pc = {solver.get_value(state['pc'])}\nq = {solver.get_value(state['q'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prime'])}\nt' = {solver.get_value(state['t_prime'])}\nr' = {solver.get_value(state['r_prime'])}")
            return
        
        print(f"> A propriedade verifica-se por k-indução (k={n}).")

In [7]:
K = 11
N = 9

In [8]:
a = 8
b = 6

In [ ]:
vars = ["pc", "r", "r_prime", "s", "s_prime", "t", "t_prime", "q"]
genTrace(vars, stronger, trans, error, K, a, b, N)
kinduction_always(vars, stronger, trans, error, K, a, b, N, safety)